<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b3ca1ea0a955f6ae5ba46094db50fcf3b1d9f154aca39feddd0ef1df82bfe75d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:


      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-01 14:20:56
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -22.50 K (-0.16%)
Current PnL: -22.90 L (-15.0%)
CY Booked + Current PnL: -9.01 L (-5.9%)
-------------------
Total profit:  1.64 L
Total loss:  -24.53 L
-------------------
Total Booked + Current PnL: 17.97 L (14.31%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.72%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.31 L (61.8%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.12,-8.70,9.55,0.02,15219.0,-15194.0,159365.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.62,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.57,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,36.0,M-SC,3.20,253.0,-0.99,0.59,10.15,OX40N,NTT,DURABLES
43,ITC,452.00,-0.20,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,44.0,X-LC,1.33,5.0,-0.12,1.38,4.06,X40,NTT,FMCG
35,ICICIGI,2252.93,0.98,6.16,13.22,20.19,24465.0,10741.0,185061.0,-16.73,49.0,X-MC,7.15,68.0,0.44,1.30,22.21,X40,ATH,INSURANCE
83,VOLTAS,1530.00,-0.73,6.89,11.98,19.69,24552.0,13203.0,204945.0,-3.56,48.0,X-MC,3.23,78.0,0.54,1.43,14.15,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,5.57,-0.20,48.93,48.63,97638.0,-407.0,199547.0,-33.63,63.0,H-SC,15.40,136.0,-0.00,1.40,11.33,XR,NTT,MISC
81,VALIANTORG,838.0,4.96,-44.99,216.58,74.15,194918.0,-73607.0,89998.0,-262.20,38.0,H-SC,18.15,149.0,-0.38,0.63,15.36,XR,NTT,CHEMICALS
76,TMPV,600.0,2.02,-13.74,64.84,42.18,154592.0,-37990.0,238420.0,-23.08,36.0,X-LC,2.28,3.0,-0.25,1.67,3.28,XY24,NTT,AUTO
70,SURYODAY,216.0,1.25,-18.50,50.96,23.03,74372.0,-33129.0,145942.0,60.09,51.0,H-SC,9.72,167.0,-0.45,1.02,44.80,XR,NTT,BANKS
64,SFL,1287.0,1.16,-40.28,112.03,26.63,175446.0,-105612.0,156606.0,18.67,36.0,M-SC,6.62,239.0,-0.60,1.10,2.34,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-6.40,-19.58,126.21,81.92,116520.0,-22476.0,92322.0,-52.84,20.0,M-SC,2.95,236.0,-0.19,0.65,9.29,XR,NTT,DURABLES
67,SIS,528.00,-2.41,-26.21,64.33,21.26,52707.0,-29100.0,81932.0,1930.05,40.0,H-SC,3.78,166.0,-0.55,0.57,10.70,OX40N,NTT,MISC
28,GREENPANEL,537.00,-2.36,-35.14,120.69,43.14,143313.0,-64333.0,118745.0,166.15,27.0,M-SC,6.34,240.0,-0.45,0.83,11.70,XY24,NTT,MISC
54,PAGEIND,51605.07,-2.27,-8.51,37.80,26.07,56624.0,-13940.0,149800.0,-32.65,28.0,X-MC,9.93,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-2.12,-36.91,114.31,35.20,92790.0,-47496.0,81174.0,6.43,20.0,X-SC,13.67,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.84,0.1,106.51,106.72,131502.0,128.0,123464.0,-50.44,40.0,H-SC,2.4,139.0,0.0,0.86,23.51,AR,ATH,MISC
37,IEX,219.00,5.57,-0.2,48.93,48.63,97638.0,-407.0,199547.0,-33.63,63.0,H-SC,15.4,136.0,-0.0,1.40,11.33,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.43,-4.71,118.19,107.91,166280.0,-6955.0,140689.0,-22.58,37.0,M-SC,10.83,216.0,-0.04,0.98,0.60,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.12,-8.70,9.55,0.02,15219.0,-15194.0,159365.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.62,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.57,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,36.0,M-SC,3.20,253.0,-0.99,0.59,10.15,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.58,-22.32,46.08,13.47,96526.0,-60192.0,209475.0,-68.12,34.0,H-SC,1.76,158.0,-0.62,1.47,6.51,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,-0.74,-26.20,78.91,32.03,76772.0,-34545.0,97290.0,-20.07,37.0,H-SC,7.70,148.0,-0.45,0.68,8.97,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.36,2.71,67.72,72.27,117861.0,4594.0,174042.0,-11.98,60.0,M-LC,3.5,99.0,0.04,1.22,9.05,XR,NTT,IT
38,INDIAMART,4810.62,0.84,0.10,106.51,106.72,131502.0,128.0,123464.0,-50.44,40.0,H-SC,2.4,139.0,0.00,0.86,23.51,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.36,2.71,67.72,72.27,117861.0,4594.0,174042.0,-11.98,60.0,M-LC,3.50,99.0,0.04,1.22,9.05,XR,NTT,IT
38,INDIAMART,4810.62,0.84,0.10,106.51,106.72,131502.0,128.0,123464.0,-50.44,40.0,H-SC,2.40,139.0,0.00,0.86,23.51,AR,ATH,MISC
37,IEX,219.00,5.57,-0.20,48.93,48.63,97638.0,-407.0,199547.0,-33.63,63.0,H-SC,15.40,136.0,-0.00,1.40,11.33,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.63,-1.24,37.33,35.63,77265.0,-2596.0,206978.0,-14.97,42.0,H-MC,3.70,119.0,-0.03,1.45,15.23,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.43,-4.71,118.19,107.91,166280.0,-6955.0,140689.0,-22.58,37.0,M-SC,10.83,216.0,-0.04,0.98,0.60,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-2.12,-36.91,114.31,35.20,92790.0,-47496.0,81174.0,6.43,20.0,X-SC,13.67,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,-2.27,-8.51,37.80,26.07,56624.0,-13940.0,149800.0,-32.65,28.0,X-MC,9.93,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-0.41,-14.41,73.45,48.45,126048.0,-28896.0,171610.0,-20.52,29.0,X-MC,7.76,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
23,DMART,5391.45,-0.86,-5.85,36.08,28.12,71474.0,-12307.0,198100.0,-18.74,33.0,X-LC,4.52,19.0,-0.17,1.39,16.39,X40N,ATH,FMCG
46,JSWINFRA,342.00,0.74,-8.24,25.57,15.23,47007.0,-16511.0,183836.0,-21.45,34.0,X-MC,6.88,66.0,-0.35,1.29,21.35,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.20,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,44.0,X-LC,1.33,5.0,-0.12,1.38,4.06,X40,NTT,FMCG
3,ACC,3906.0,0.01,-22.22,111.06,64.15,205539.0,-52881.0,185070.0,-54.57,49.0,X-SC,1.57,82.0,-0.26,1.30,3.81,XY24,BTT,CEMENT
15,CAMS,4762.0,1.14,2.58,21.65,24.79,55934.0,6501.0,258357.0,-2.14,52.0,X-SC,1.88,86.0,0.12,1.81,26.32,X40N,NTT,MISC
66,SIEMENS,4671.5,0.15,-11.29,41.48,25.51,68479.0,-21005.0,165090.0,1.35,64.0,H-LC,2.03,49.0,-0.31,1.16,21.66,AR,ATH,ELECTRICAL
36,ICICIPRULI,790.0,0.19,2.93,27.23,30.97,49538.0,5186.0,181924.0,-21.26,55.0,X-MC,2.12,75.0,0.10,1.27,15.86,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-2.12,-36.91,114.31,35.20,92790.0,-47496.0,81174.0,6.43,20.0,X-SC,13.67,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.41,-14.41,73.45,48.45,126048.0,-28896.0,171610.0,-20.52,29.0,X-MC,7.76,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-2.27,-8.51,37.80,26.07,56624.0,-13940.0,149800.0,-32.65,28.0,X-MC,9.93,55.0,-0.25,1.05,0.00,X40,ATH,APPARELS
55,PGHH,17907.65,0.34,-4.39,39.90,33.76,76608.0,-8820.0,192000.0,-32.51,35.0,X-MC,4.06,57.0,-0.12,1.34,0.58,X40,ATH,FMCG
19,COLPAL,3726.84,-0.55,-18.11,72.81,41.51,157022.0,-47705.0,215660.0,-2.56,40.0,X-MC,7.21,61.0,-0.30,1.51,1.20,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.43,-27.81,97.87,42.84,45927.0,-18079.0,46927.0,-52.62,42.0,X-SC,37.51,83.0,-0.39,0.33,2.11,XY24,NTT,MISC
60,RELAXO,1176.00,0.04,-46.98,187.25,52.31,144121.0,-68193.0,76967.0,-43.57,37.0,X-SC,4.80,91.0,-0.47,0.54,3.02,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-2.12,-36.91,114.31,35.20,92790.0,-47496.0,81174.0,6.43,20.0,X-SC,13.67,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,0.31,-5.94,29.26,21.58,38569.0,-8325.0,131814.0,-52.56,37.0,X-MC,6.21,56.0,-0.22,0.92,16.70,X40N,ATH,IT
52,MEDANTA,1486.00,-0.48,3.54,19.84,24.08,26078.0,4490.0,131440.0,-4.85,46.0,X-SC,5.94,89.0,0.17,0.92,23.85,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.03,-14.75,39.92,19.28,117734.0,-51031.0,294925.0,-25.57,58.0,X-LC,7.11,1.0,-0.43,2.06,9.03,X40,ATH,IT
41,INFY,2275.00,0.41,6.64,45.23,54.87,150916.0,20776.0,333664.0,-17.09,62.0,X-LC,2.35,2.0,0.14,2.34,15.47,X40,BTT,IT
76,TMPV,600.00,2.02,-13.74,64.84,42.18,154592.0,-37990.0,238420.0,-23.08,36.0,X-LC,2.28,3.0,-0.25,1.67,3.28,XY24,NTT,AUTO
82,VBL,671.64,0.44,-2.30,38.87,35.67,119941.0,-7273.0,308569.0,-13.89,63.0,X-LC,5.48,4.0,-0.06,2.16,11.23,X40N,ATH,FMCG
43,ITC,452.00,-0.20,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,44.0,X-LC,1.33,5.0,-0.12,1.38,4.06,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.69,-14.36,112.14,81.67,90091.0,-13472.0,80338.0,7222.22,46.0,L-SC,19.37,271.0,-0.15,0.56,55.58,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.45,-11.87,58.33,39.54,52004.0,-12005.0,89154.0,-31.03,66.0,M-SC,6.69,220.0,-0.23,0.62,25.72,AR,ATH,AUTO
51,MASFIN,398.61,0.24,-5.22,28.77,22.05,26717.0,-5115.0,92865.0,-18.25,50.0,H-SC,6.81,164.0,-0.19,0.65,34.99,XR,ATH,FINANCE
70,SURYODAY,216.00,1.25,-18.50,50.96,23.03,74372.0,-33129.0,145942.0,60.09,51.0,H-SC,9.72,167.0,-0.45,1.02,44.80,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.12,-8.70,9.55,0.02,15219.0,-15194.0,159365.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.62,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.12,-8.70,9.55,0.02,15219.0,-15194.0,159365.0,143.30,67.0,M-SC,11.74,234.0,-1.00,1.12,38.62,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.54,-0.07,21.03,20.95,52908.0,-185.0,251583.0,-1.05,71.0,X-LC,15.75,31.0,-0.00,1.76,35.54,X40,ATH,PAINTS
50,LTIM,7201.88,0.73,12.27,17.28,31.66,47752.0,30194.0,276345.0,8.03,68.0,H-LC,13.97,43.0,0.63,1.93,53.33,X200,ATH,IT
40,INDUSINDBK,1800.00,-1.37,-37.48,112.56,32.89,54330.0,-28938.0,48268.0,-696.43,60.0,L-MC,5.80,259.0,-0.53,0.34,32.93,XR,NTT,BANKS
32,HCLTECH,1922.01,0.87,6.33,17.31,24.73,44526.0,15305.0,257229.0,11.06,65.0,X-LC,4.88,7.0,0.34,1.80,22.53,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.97
1,25,44.86
2,50,75.78


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.43
MC    29.13
LC    25.47
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.74
X40      21.26
X40N     12.04
XR        9.62
XY25      9.08
AR        9.05
OX40N     7.58
X200      1.93
SR        0.98
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.88
X-MC    22.66
X-LC    20.18
M-SC    11.87
X-SC     8.28
H-MC     4.75
H-LC     3.09
L-SC     1.40
M-MC     1.38
M-LC     1.22
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.84,-4.56,38.37
IT,13.85,-14.17,73.10
FINANCE,9.45,-14.81,63.32
MISC,7.41,-22.71,72.98
ELECTRICAL,6.01,-9.36,49.42
PAINTS,5.93,-9.05,25.25
INSURANCE,4.41,-0.36,35.73
PHARMA,3.90,-0.11,32.16
AUTO,3.40,-18.19,68.87


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3074731.0,21
XR,1288950.0,13
AR,1278974.0,10
X40,1000753.0,14
X40N,729296.0,9
OX40N,709428.0,10
XY25,343079.0,6
SR,280554.0,2
MH,77223.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3566042.0,25
M-SC,1365310.0,15
X-MC,1243110.0,16
X-LC,879780.0,11
X-SC,738428.0,8
H-MC,399953.0,3
L-SC,261925.0,3
M-LC,117861.0,1
H-LC,116231.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1221722.0      6
           AR         880339.0      5
M-SC       XY24       786465.0      6
H-SC       XR         776626.0      7
X-MC       X40        441540.0      7
X-LC       X40        389818.0      5
X-MC       XY24       373604.0      3
H-SC       OX40N      329578.0      4
M-SC       OX40N      292288.0      5
X-SC       X40N       291489.0      3
H-SC       SR         280554.0      2
X-SC       XY24       277544.0      3
X-MC       X40N       246392.0      4
X-LC       XY24       224812.0      2
H-MC       AR         209369.0      2
X-LC       X40N       191415.0      2
H-MC       XY24       190584.0      1
X-MC       XY25       181574.0      2
L-SC       XR         174363.0      2
X-SC       X40        169395.0      2
M-SC       XR         165770.0      2
           AR         120787.0      2
M-LC       XR         117861.0      1
L-SC       OX40N       87562.0      1
H-SC       MH          77223.0      1
X-LC       XY25        73735.0      2
H-LC       AR          68479.0      1
L-MC       XR          54330.0      1
M-MC       XY25        48509.0      1
H-LC       X200        47752.0      1
L-LC       XY25        39261.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
